In [4]:
import numpy as np
from numpy import linalg as LA
import  math
import numba
from numba import jit


@jit(nopython=True) 
def fermi(kT,E):
    N = 1/(np.exp(E/kT) + 1)
    return N



@jit(nopython=True) 
def kmesh_preparation(cell_vec):
    #reciprocal vectors and and kmesh for integration
    rec_vec = np.zeros((3,3))
    k_vec  = np.zeros((num_kpoints,3))

    rec_vec[0]  = (2 * np.pi / np.linalg.det(cell_vec)) * np.cross(cell_vec[1], cell_vec[2])
    rec_vec[1]  = (2 * np.pi / np.linalg.det(cell_vec)) * np.cross(cell_vec[2], cell_vec[0])
    rec_vec[2]  = (2 * np.pi / np.linalg.det(cell_vec)) * np.cross(cell_vec[0], cell_vec[1])

    idx = 0
    for i in range(kmesh[0]):
        for j in range(kmesh[1]):
            for k in range(kmesh[2]):
                for z in range(3):
                    k_vec[idx,z] = (rec_vec[0,z] * i/ kmesh[0]) + (rec_vec[1,z] * j / kmesh[1]) + (rec_vec[2,z] * k / kmesh[2])

                idx+=1

    return k_vec 


@jit(nopython=True) 
def calc_hamK(num_orb, num_kpoints, n_min, n_max, cell_vec, k_vec, Ham_R):
    #Fourier transformation  of Hamiltonian
    Ham_K = np.zeros((num_kpoints, num_orb, num_orb), dtype=np.complex128)

    for i in range(n_min[0], n_max[0]):
        for j in range(n_min[1], n_max[1]):
            for k in range(n_min[2], n_max[2]+1):
                r = i * cell_vec[0] + j * cell_vec[1] + k * cell_vec[2]
                t = (k_vec @ r).reshape(num_kpoints, 1, 1)
                t = np.exp(-1j * t)
                rhs = np.ascontiguousarray(Ham_R[i + n_max[0], j + n_max[1], k + n_max[2],:,:])
                Ham_K += t * rhs

    return Ham_K




@jit(nopython=True) 
def polarization_operator(num_orb, kmesh, HamK):
    num_kpoints = kmesh[0] * kmesh[1] * kmesh[2]
    weight = 1/(num_kpoints)
    
    rpa_pol = np.zeros((kmesh[0], kmesh[1], kmesh[2]), dtype=np.complex128)
        
    #smearing in eV
    delta = 0.004  # 50K
    kT = 0.004  # 50K
 
    for q1 in range(kmesh[0]):
        for q2 in range(kmesh[1]):
            for q3 in range(kmesh[2]): #q
                q  =  (q1 * kmesh[1] * kmesh[2])  + (q2 * kmesh[2]) + q3
                print(q)
        
                for k1 in range(kmesh[0]):
                    for k2 in range(kmesh[1]):
                        for k3 in range(kmesh[2]): #k
                            k = (k1 * kmesh[1] * kmesh[2])  + (k2 * kmesh[2]) + k3  # k
                            w_k, v_k = LA.eigh(HamK[k])
                            
                            
                            kq1 = (q1 + k1)%kmesh[0] 
                            kq2 = (q2 + k2)%kmesh[1] 
                            kq3 = (q3 + k2)%kmesh[2] #k+q
                            
                            kq = (kq1 * kmesh[1] * kmesh[2]) + (kq2 * kmesh[2]) + kq3  #k+q 
                            w_kq, v_kq = LA.eigh(HamK[kq])
                            
                            #rpa polarisation
                            for n in range(num_orb):
                                for m in range(num_orb):
                                    rpa_pol[q1, q2, q3] += 2 * weight* (fermi(kT, w_k[n]) - fermi(kT, w_kq[m])) \
                                    * np.dot(v_k[n], v_kq[m])**2 / (w_k[n] - w_kq[m] - 1j*delta) 
                                                                              
    
    
    return rpa_pol



if __name__ == '__main__':
    
    kmesh = np.array([3,3,3])
    num_kpoints = kmesh[0] * kmesh[1] * kmesh[2]
    weight = 1/(num_kpoints)
    kT = 100

    e_fermi = 0.1520 # eV

    cell_vec = np.array([[-1.742087135,   1.742087135,   3.467599447],
                        [1.742087135,  -1.742087135,   3.467599447], 
                        [1.742087135,   1.742087135,  -3.467599447]])

    k_vec = kmesh_preparation(cell_vec)

    with open('tb_model.dat') as fp:
        rows = (line.split() for line in fp)
        data = [([int(u) for u in row[:3]], [int(u) for u in row[3:5]],
                 [float(u) for u in row[5:]]) for row in rows]

    # [N, 3] vectors, [N, 2] orbitals, [N, 2] hamiltonian
    vecs, orbs, ham_values = map(np.array, zip(*data))
    ham_values = ham_values.astype('f8').view('c16').ravel()  # View as complex [N]

    num_orb = max(orbs[:,0])
    n_min, n_max = vecs.min(0), vecs.max(0)  # [3]
    n_size = n_max - n_min + 1  # Plus 1 for 0th

    Ham_R = np.zeros((*n_size, num_orb, num_orb), dtype='c16')
    Ham_R[(*(vecs + n_max).T, *(orbs.T - 1))] = ham_values

    Ham_K = calc_hamK(num_orb, num_kpoints, n_min, n_max, cell_vec, k_vec, Ham_R)
    
    print("Ham_K is ready!")
    
    rpa_pol =  polarization_operator(num_orb, kmesh, Ham_K)
    print(rpa_pol.shape)

Ham_K is ready!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
(3, 3, 3)
